In [1]:
!rm *.csv
!rm cnn*
!wget http://smartheatmap.ddns.net/dist/p.csv
!wget http://smartheatmap.ddns.net/dist/n.csv
!wget http://smartheatmap.ddns.net/dist/w2v.zip
!unzip w2v.zip

rm: cannot remove 'cnn*': No such file or directory
--2019-07-18 15:54:40--  http://smartheatmap.ddns.net/dist/p.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27630235 (26M) [text/csv]
Saving to: ‘p.csv’

p.csv               100%[===================>]  26.35M  11.7MB/s    in 2.2s    

2019-07-18 15:54:43 (11.7 MB/s) - ‘p.csv’ saved [27630235/27630235]

--2019-07-18 15:54:44--  http://smartheatmap.ddns.net/dist/n.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26037206 (25M) [text/csv]
Saving to: ‘n.csv’

n.csv               100%[===================>]  24.83M  11.8MB/s    in 2.1s    

2019-07-18 15:54:46 (11.8 MB/s) - ‘n.csv’ saved [26

In [0]:
import pandas as pd
import numpy as np

data_positive = pd.read_csv('p.csv', sep=';', error_bad_lines=False, usecols=['ttext'])
data_negative = pd.read_csv('n.csv', sep=';', error_bad_lines=False, usecols=['ttext'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['ttext'].values[:sample_size],
                           data_negative['ttext'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [0]:
import re

def clear_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [clear_text(t) for t in raw_data]


In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

SENTENCE_LENGTH = 26
NUM = 100000

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

def to_seq(x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
x_train = to_seq(x_train)
x_test = to_seq(x_test)

Using TensorFlow backend.


KeyboardInterrupt: ignored

In [0]:
from gensim.models import Word2Vec

wvmodel = Word2Vec.load("w2v/tweets_model.w2v")

DIM = wvmodel.vector_size 
embedding_matrix = np.zeros((NUM, DIM))

for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in wvmodel.wv.vocab.keys():
        embedding_matrix[i] = wvmodel.wv[word]

In [0]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

m_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
m_embed = Embedding(NUM, 
                    DIM, 
                    input_length=SENTENCE_LENGTH,
                    weights=[embedding_matrix], 
                    trainable=True)(m_input)
# DIM=1000
# m_embed = Embedding(NUM, 
#                     DIM, 
#                     input_length=SENTENCE_LENGTH)(m_input)

In [0]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout, LSTM, CuDNNLSTM
from keras.models import Model, Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.regularizers import l1

# x = Dropout(0.2)(m_embed)

# conv_branches = []
# for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
#     for i in range(filters_count):
#         i = Conv1D(filters=1, 
#                    kernel_size=size, 
#                    padding='valid', 
#                    activation='relu')(x)
#         i = GlobalMaxPooling1D()(i)
#         conv_branches.append(i)

# x = concatenate(conv_branches, 
#                 axis=1)

# x = Dropout(0.2)(x)
# x = Dense(30, activation='relu')(x)
# x = Dense(1)(x)

# m_output = Activation('sigmoid')(x)

model = Sequential()
model.add(Embedding(NUM, 
                    DIM, 
                    input_length=SENTENCE_LENGTH,
                    weights=[embedding_matrix], 
                    trainable=False))
model.add(Dropout(0.3))

model.add(CuDNNLSTM(200, return_sequences=True))
model.add(CuDNNLSTM(10))

model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model = Model(inputs=[m_input], 
#               outputs=[m_output])

In [0]:
from keras import backend as K


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy',
                       precision, 
                       recall, 
                       f1])

In [0]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("cnn-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', 
                             save_best_only=True, 
                             mode='max', 
                             period=1)

history = model.fit(x_train, 
                    y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_split=0.25, 
                    callbacks = [checkpoint])

In [0]:
model.predict(to_seq(["Зачем мне подрывать и без того не самую безупречную репутацию кураторов?",
                       "сломанная лавочка",
                       "Ну это же логично, что я дождался, пока ты прочтешь и удалил",
                       "трансгендер на улице",
                       "США",
                       "СССР",
                       "фашист",
                       "на улице поставили хреновую красивую скамейку",
                       "на улице хрено поставили красивую скамейку",
                       "на улице Ленина поставили хреновую скамейку",
                       "красивую скамейку",
                       "мне нравится эта хрень, но будет нереально круто",
                       "Жить конечно тут наверное нет",
                       "Очень-очень красиво, сказочно, солнечно, жарко, волшебно....можно много слов написать, и все они будут про это не забываемое место. Я  не знаю у кого как. Но я очень люблю Сочи-Адлер и люблю тут отдыхать с детьми и одна и с подругами и вообще. Жить конечно тут наверное нет. Но летом сюда-это точно. На все лето, сюда☺☺☺🤩🤩🤩🤩🤩",
                       "эта скамейка очень красивая",
                       "сегодня было грустно",
                       "эта скамейка полная чушь"]))